In [2]:
import tensorflow as tf

In [3]:
print(tf.get_default_graph())
g = tf.Graph()
print(g)
a = tf.constant(5)
print(a.graph is g)
print(a.graph is tf.get_default_graph())
print("------------------------")
g1 = tf.get_default_graph()
g2 = tf.Graph()
print(g1 is tf.get_default_graph())
with g2.as_default():
    print(g1 is tf.get_default_graph())
print(g1 is tf.get_default_graph())


False
True
------------------------
True
False
True


In [4]:
a = tf.constant(6)
b = tf.constant(6)
c = tf.multiply(a, b)
d = tf.add(a, b)
e = tf.subtract(d, c)
f = tf.add(c, d)
with tf.Session() as sess:
    fetches = [a,b,c,d,e,f]
    outs = sess.run(fetches)
print("outs = {}".format(outs))
print(type(outs[0]))


outs = [6, 6, 36, 12, -24, 48]
<class 'numpy.int32'>


In [5]:
c = tf.constant(4.0, dtype=tf.float64)
print(c)
print(c.dtype)

Tensor("Const_3:0", shape=(), dtype=float64)
<dtype: 'float64'>


In [6]:
x = tf.constant([1,2,3],name='x',dtype=tf.float32)
print(x.dtype)
x = tf.cast(x,tf.int64)
print(x.dtype)

<dtype: 'float32'>
<dtype: 'int64'>


In [7]:
import numpy as np

In [9]:
c = tf.constant([[1,2,3],[4,5,6]])
print("python list input:{}".format(c.get_shape()))
c = tf.constant(np.array([[[1,2,3],[4,5,6]],[[1,1,1],[2,2,2]]]))
print("3d numpy array input:{}".format(c.get_shape()))

python list input:(2, 3)
3d numpy array input:(2, 2, 3)


In [10]:
sess = tf.InteractiveSession()
c = tf.linspace(0.0,4.0,5)
print("the content of 'c':\n {}\n".format(c.eval()))
sess.close()

the content of 'c':
 [0. 1. 2. 3. 4.]



In [13]:
A = tf.constant([ [1,2,3],[4,5,6] ])
print(A.get_shape())
x = tf.constant([1,0,1])
print(x.get_shape())

(2, 3)
(3,)


In [14]:
x = tf.expand_dims(x,1)
print(x.get_shape())
b = tf.matmul(A,x)
sess = tf.InteractiveSession()
print("matmul result:\n{}".format(b.eval()))
sess.close()

(3, 1)
matmul result:
[[ 4]
 [10]]


In [16]:
with tf.Graph().as_default():
    c1 = tf.constant(4,dtype=tf.float64,name="c")
    c2 = tf.constant(4,dtype=tf.int32,name="c")
print(c1.name)
print(c2.name)

c:0
c_1:0


In [19]:
with tf.Graph().as_default():
    c1 = tf.constant(4,dtype=tf.float64,name="c")
    with tf.name_scope("prefix_name"):
        c2 = tf.constant(4,dtype=tf.int32,name="c")
        c3 = tf.constant(4,dtype=tf.float64,name="c")
print(c1.name)
print(c2.name)
print(c3.name)

c:0
prefix_name/c:0
prefix_name/c_1:0


In [20]:
init_val = tf.random_normal((1,5),0,1)
var = tf.Variable(init_val,name="var")
print("pre run:\n{}".format(var))
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    post_var = sess.run(var)
print("\npost run:\n{}".format(post_var))

pre run:
<tf.Variable 'var:0' shape=(1, 5) dtype=float32_ref>

post run:
[[ 0.10503287 -0.25116864 -0.35292348 -0.07164145  1.3167444 ]]


In [24]:
x_data = np.random.randn(5,10)
w_data = np.random.randn(10,1)
with tf.Graph().as_default():
    x = tf.placeholder(tf.float32,shape=(5,10))
    w = tf.placeholder(tf.float32,shape=(10,1))
    b = tf.fill((5,1),-1.)
    xw = tf.matmul(x,w)
    xwb = xw + b
    s = tf.reduce_max(xwb)
    with tf.Session() as sess:
        outs = sess.run(s,feed_dict={x:x_data,w:w_data})
print("outs = {}".format(outs))

outs = 4.063295364379883


In [29]:
x = tf.placeholder(tf.float32,shape = [None,3])
y_true = tf.placeholder(tf.float32,shape=None)
w = tf.Variable([[0,0,0]],dtype=tf.float32,name="weights")
b = tf.Variable(0,dtype=tf.float32,name="bias")
y_pred = tf.matmul(w,tf.transpose(x)) + b
print(y_pred)

Tensor("add_5:0", shape=(1, ?), dtype=float32)


In [33]:
import matplotlib.pyplot as plt
x_data = np.random.randn(2000,3)
w_real = [0.3,0.5,0.1]
b_real = -0.2
noise = np.random.randn(1,2000)*0.1
y_data = np.matmul(w_real,x_data.T)+b_real+noise
plt.show()

In [37]:
NUM_STEPS = 10
g = tf.Graph()
wb_ = []
with g.as_default():
    x = tf.placeholder(tf.float32,shape=[None,3])
    y_true = tf.placeholder(tf.float32,shape=None)
    with tf.name_scope("inference") as scope:
        w = tf.Variable([[0,0,0]],dtype=tf.float32,name='weights')
        b = tf.Variable(0,dtype=tf.float32,name='bias')
        y_pred = tf.matmul(w,tf.transpose(x))+b
    with tf.name_scope("loss") as scope:
        loss = tf.reduce_mean(tf.square(y_true-y_pred))
    with tf.name_scope("train") as scope:
        learning_rate = 0.5
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(loss)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for step in range(NUM_STEPS):
            sess.run(train,{x:x_data,y_true:y_data})
            if(step%5 == 0):
                print(step,sess.run([w,b]))
                wb_.append(sess.run([w,b]))
        print(10,sess.run([w,b]))

0 [array([[0.29074353, 0.4890837 , 0.10200132]], dtype=float32), -0.18895584]
5 [array([[0.298139  , 0.49980703, 0.09923898]], dtype=float32), -0.2013013]
10 [array([[0.298139  , 0.49980703, 0.09923897]], dtype=float32), -0.2013013]


In [40]:
N = 20000
def sigmoid(x):
    return 1/(1+np.exp(-x))
x_data = np.random.randn(N,3)
w_real = [0.3,0.5,0.1]
b_real = -0.2
wxb = np.matmul(w_real,x_data.T)+b_real
y_data_pre_noise = sigmoid(wxb)
y_data = np.random.binomial(1,y_data_pre_noise)

In [48]:
NUM_STEPS = 50
g = tf.Graph()
wb_ = []
with g.as_default():
    x = tf.placeholder(tf.float32,shape=[None,3])
    y_true = tf.placeholder(tf.float32,shape=None)
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]],dtype=tf.float32,name='weights')
        b = tf.Variable(0,dtype=tf.float32,name='bias')
        y_pred = tf.matmul(w,tf.transpose(x)) + b
    with tf.name_scope('loss') as scope:
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true,logits=y_pred) 
        loss = tf.reduce_mean(loss)
    with tf.name_scope('train') as scope:
        learning_rate = 0.5
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(loss)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for step in range(NUM_STEPS):
            sess.run(train,{x:x_data,y_true:y_data})
            if(step%5 == 0):
                print(step,sess.run([w,b]))
                wb_.append(sess.run([w,b]))

        print(50,sess.run([w,b]))

0 [array([[0.03357116, 0.05859274, 0.01339402]], dtype=float32), -0.027149983]
5 [array([[0.14799456, 0.25856963, 0.05810415]], dtype=float32), -0.11994014]
10 [array([[0.2087286 , 0.36499316, 0.08098447]], dtype=float32), -0.16929369]
15 [array([[0.24250868, 0.42434877, 0.09329957]], dtype=float32), -0.1967056]
20 [array([[0.26187885, 0.4584771 , 0.10015748]], dtype=float32), -0.21237402]
25 [array([[0.27318928, 0.47845665, 0.10405801]], dtype=float32), -0.2214872]
30 [array([[0.27986455, 0.49027672, 0.10630605]], dtype=float32), -0.22684366]
35 [array([[0.28382927, 0.4973127 , 0.10761277]], dtype=float32), -0.23001237]
40 [array([[0.286193 , 0.501516 , 0.1083767]], dtype=float32), -0.23189443]
45 [array([[0.28760546, 0.5040322 , 0.10882507]], dtype=float32), -0.23301515]
50 [array([[0.28831437, 0.50529706, 0.10904673]], dtype=float32), -0.23357598]
